# Import Required Libraries
Import the necessary libraries, including pythreejs, numpy, scipy, and ipywidgets.

In [1]:
from pythreejs import *
from IPython.display import display
import ipywidgets as widgets
import numpy as np
import time
from scipy.spatial.transform import Rotation as R
import asyncio

# Define Geometry Classes
Define the Plane and Cube classes for creating geometric objects.

In [2]:
class Plane(Mesh):
    def __init__(self, origin=[0, 0, 0], normal=[0, 1, 0], size=10, color='gray', opacity=1.0, **kwargs):
        geometry = PlaneGeometry(width=size, height=size)
        material = MeshStandardMaterial(
            color=color, 
            transparent=True, 
            opacity=opacity, 
            side='DoubleSide'
        )
        super().__init__(geometry=geometry, material=material, **kwargs)
        self.receiveShadow = True
        
        # Rotate the plane to the normal vector
        z = np.array([0, 0, 1])  # PlaneGeometry default normal
        axis = np.cross(z, normal)
        angle = np.arccos(np.dot(z, normal))  # Angle between vectors

        if np.linalg.norm(axis) > 1e-6:  # Avoid division by zero if vectors are parallel
            axis = axis / np.linalg.norm(axis)
            self.quaternion = list(np.append(axis * np.sin(angle / 2), np.cos(angle / 2)))  # Quaternion rotation 

        self.origin = np.array(origin)
        self.normal = np.array(normal)
        

class Cube(Mesh):
    def __init__(self, width=1.0, height=1.0, depth=1.0, color='green', position=[0, 0, 0], rotation=[0, 0, 0]):
        vertices, self.faces, self.edges, self.tets = self.generate_cube(width, height, depth)
        
        geometry = BufferGeometry(
            attributes={
                'position': BufferAttribute(array=vertices.copy(), normalized=False),
            },
            index=BufferAttribute(array=self.faces.copy().ravel(), normalized=False)
        )
        
        material = MeshStandardMaterial(
            color=color,  # RGB color of the material
            flatShading=True,
            wireframe=False,
        )
        super().__init__(geometry=geometry, material=material)
        
        self.castShadow = True

        # Transformation
        self.init_rotation = R.from_euler('XYZ', rotation, degrees=True)
        self.init_position = (np.array(position, dtype=np.float32) +  self.init_rotation.apply(self.geometry.attributes['position'].array)).copy()
        self.reset()


        # self.tets = tets.copy()
        # self.rest_volume = np.einsum('ij,ij->i', np.cross(self.init_position[tets[:, 1]].copy() - self.init_position[tets[:, 0]].copy(), 
        #                                                   self.init_position[tets[:, 2]].copy() - self.init_position[tets[:, 0]].copy()
        #                                                   ), self.init_position[tets[:, 3]].copy() - self.init_position[tets[:, 0]].copy()) / 6.0
        # swap_idx = np.where(self.rest_volume < 0)[0]
        # self.tets[swap_idx[..., None], [1, 2]] = self.tets[swap_idx[..., None], [2, 1]]
        # self.rest_volume = np.abs(self.rest_volume)
    
    def reset(self):
        self.geometry.attributes['position'].array = self.init_position.copy()
        
    
    def generate_cube(self, width=1.0, height=1.0, depth=1.0):
        """Generate vertices and faces for a box."""
        vertices = np.array([
            [-width/2, -height/2, -depth/2],  # 0 - Back Bottom Left
            [ width/2, -height/2, -depth/2],  # 1 - Back Bottom Right
            [ width/2,  height/2, -depth/2],  # 2 - Back Top Right
            [-width/2,  height/2, -depth/2],  # 3 - Back Top Left
            [-width/2, -height/2,  depth/2],  # 4 - Front Bottom Left
            [ width/2, -height/2,  depth/2],  # 5 - Front Bottom Right
            [ width/2,  height/2,  depth/2],  # 6 - Front Top Right
            [-width/2,  height/2,  depth/2],  # 7 - Front Top Left
            [0, 0, 0] # 8
        ], dtype=np.float32)

        # Define 12 triangles (6 faces x 2 triangles per face)
        faces = np.array([
            [4, 5, 6], [4, 6, 7],  # Front (+Z)
            [1, 0, 3], [1, 3, 2],  # Back (-Z)
            [5, 1, 2], [5, 2, 6],  # Right (+X)
            [0, 4, 7], [0, 7, 3],  # Left (-X)
            [3, 7, 6], [3, 6, 2],  # Top (+Y)
            [0, 1, 5], [0, 5, 4]   # Bottom (-Y)
        ], dtype=np.uint32)
        
        
        edges = np.array([
            [0, 1], [1, 2], [2, 3], [3, 0], # Back face
            [4, 5], [5, 6], [6, 7], [7, 4], # Front face
            [0, 4], [1, 5], [2, 6], [3, 7], # Connect front and back faces
            [0, 2], [1, 3], [4, 6], [5, 7], # Connect top and bottom faces
            [1, 6], [2, 5], [0, 7], [3, 4], # Connect front and back faces
            [0, 5], [1, 4], [2, 7], [3, 6], # Connect front and back faces
        ], dtype=np.uint32)
        
        
        tet_indices = np.array([
            [8, 0, 1, 2], [8, 0, 2, 3], # Back face
            [8, 4, 5, 6], [8, 4, 6, 7], # Front face
            [8, 0, 4, 5], [8, 0, 5, 1], # Connect front and back faces
            [8, 2, 6, 7], [8, 2, 7, 3], # Connect front
            [8, 5, 1, 6], [8, 1, 2, 6], # Connect front
            [8, 0, 4, 7], [8, 0, 7, 3], # Connect back
            
        ], dtype=np.uint32)
        
        return vertices, faces, edges, tet_indices

# Physical Object

In [3]:
class PhysicalObject:
    def __init__(self, mesh, is_static=True, friction=0.6, restitution=0.6):
        self.mesh = mesh
        
        self.edges = mesh.edges if hasattr(mesh, 'edges') else None
        if self.edges is not None:
            self.compute_rest_length()
        
        self.faces = mesh.faces if hasattr(mesh, 'faces') else None
        
        self.is_static = is_static
        
        self.currPos = None
        self.currVel = None
        self.prevPos = None
        self.prevVel = None       
        self.reset()
        
        self.inv_mass = 1.0 if not is_static else 0.0
        self.friction = friction
        self.restitution = restitution
    
    def reset(self):
        if self.is_static:
            return
        self.mesh.reset()
        
        self.currPos = self.mesh.init_position.copy()
        self.currVel = np.zeros_like(self.mesh.init_position)
        self.prevPos = self.mesh.init_position.copy()
        self.prevVel = np.zeros_like(self.mesh.init_position)
    
    def compute_rest_length(self,):
        self.rest_length = np.linalg.norm(self.mesh.init_position[self.edges[:, 0]].copy() - self.mesh.init_position[self.edges[:, 1]].copy(), axis=1)
    
    def integrate(self, g, dt):
        pass
        
    
    def update_mesh(self):
        self.mesh.geometry.attributes['position'].array = self.currPos.copy()

# Constraints

In [4]:
# Define Constraint Classes
# Define the Constraint, DistanceConstraint, JointPositionConstraint, AttachmentConstraint, PlaneCollisionConstraint, and DynamicCollisionConstraint classes.

class Constraint:
    def __init__(self, compliance, lambda_):
        self.compliance = compliance
        self.lambda_ = lambda_
        
    def solve(self):
        pass

class DistanceConstraint(Constraint):
    def __init__(self, body1, id1, body2, id2, rest_length, compliance=0.000001, lambda_=0.0):
        self.body1 = body1
        self.id1 = id1
        self.w1 = self.body1.inv_mass
        
        self.body2 = body2
        self.id2 = id2
        self.w2 = self.body2.inv_mass
        
        self.rest_length = rest_length
        self.compliance = compliance
        self.lambda_ = lambda_
        
    def solve(self, h):
        x1, x2 = self.body1.currPos[self.id1], self.body2.currPos[self.id2]
        
        n = x1 - x2
        d = np.linalg.norm(n)
        C = d - self.rest_length
        if d < 1e-6:
            return
        
        alpha = self.compliance / h / h
        dlambda = - (C + alpha * self.lambda_) / (self.w1 + self.w2 + alpha)
        self.lambda_ += dlambda
        
        dx = dlambda * n / d
        
        self.body1.currPos[self.id1] += self.w1 * dx
        self.body2.currPos[self.id2] -= self.w2 * dx
          

class PointPlaneDistanceConstraint(Constraint):
    def __init__(self, body1, id1, body2, plane_id, compliance=0.0, lambda_=0.0):
        self.body1 = body1
        self.id1 = id1
        self.w1 = body1.inv_mass
        
        self.body2 = body2
        self.face_id = 2 * plane_id
        """
        4 -- 3
        |  / |
        | /  |
        1 -- 2             
        """        
        self.ids2 = list(self.body2.faces[self.face_id]).append(self.body2.faces[self.face_id+1][-1])
        self.w2 = body2.inv_mass
        
        self.rest_length = 0
        self.compliance = compliance
        self.lambda_ = lambda_

    def solve(self, h):
        p  = self.body1.currPos[self.id1]     
        q1 = self.body2.currPos[self.ids2[0]]
        q2 = self.body2.currPos[self.ids2[1]]
        q3 = self.body2.currPos[self.ids2[2]]
        q4 = self.body2.currPos[self.ids2[3]]
        
        e1 = q2 - q1
        e2 = q4 - q1
        v = p - q1
        
        n = np.cross(e1, e2)
        d = np.dot(n, v) / np.linalg.norm(n)
        if d >=0: 
            print('need debug1')
            return

        C = d

        # Dynamic Plane
        b1, b2, b3, b4 = 0, 0, 0, 0
        if self.w2 != 0: # non - static object

            s = np.dot(e1, v) / np.dot(e1, e1)
            t = np.dot(e2, v) / np.dot(e2, e2)

            b1 = (1 - s) * (1 - t)
            b2 = s * (1 - t)
            b3 = s * t
            b4 = (1 - s) * t

            p_proj = b1 * q1 + b2 * q2 + b3 * q3 + b4 * q4
            n = p - p_proj
            if np.linalg.norm(n) - d > 1e-6:
                print('need debug2')
                return
    
        dC_p = n
        dC_q1 = -n * b1
        dC_q2 = -n * b2
        dC_q3 = -n * b3
        dC_q4 = -n * b4
        
        ## Compute the Lagrange Multiplier (XPBD)
        alpha = self.compliance / h / h
        dlambda = - (C + alpha * self.lambda_) / (self.w1 * 1 + self.w2 * (b1**2 + b2**2 + b3**2 + b4**2) + alpha)
        self.lambda_ += dlambda
        
        dp = dlambda * dC_p
        dq1 = dlambda * dC_q1
        dq2 = dlambda * dC_q2
        dq3 = dlambda * dC_q3
        dq4 = dlambda * dC_q4    
        
        self.body1.currPos[self.id1] += self.w1 * dp
        self.body2.currPos[self.face_id[0]] += self.w2 * b1 * dq1
        self.body2.currPos[self.face_id[1]] += self.w2 * b2 * dq2
        self.body2.currPos[self.face_id[2]] += self.w2 * b3 * dq3
        self.body2.currPos[self.face_id[3]] += self.w2 * b4 * dq4
        

class EdgeEdgeDistanceConstraint(Constraint):
    def __init__(self, body1, edge_id1, body2, edge_id2, rest_length, compliance=0.0, lambda_=0.0):
        self.body1 = body1
        self.edge_id1 = self.body1.edges[edge_id1]
        self.w1 = 1 / self.body1.mass
        
        self.body2 = body2
        self.edge_id2 = self.body2.edges[edge_id2]
        self.w2 = 1 / self.body2.mass
        
        self.rest_length = rest_length
        self.compliance = compliance
        self.lambda_ = lambda_
    
    def solve(self, h):
        p1, p2 = self.body1.currPos[self.edge_id1[0]], self.body1.currPos[self.edge_id1[1]]
        q1, q2 = self.body2.currPos[self.edge_id2[0]], self.body2.currPos[self.edge_id2[1]]
        

class AttachmentConstraint(Constraint):
    def __init__(self, body, id, rest_position, compliance=0.0, lambda_=0.0):
        self.body = body
        self.id = id
        self.w = body.inv_mass
        
        self.rest_position = rest_position
        self.compliance = compliance
        self.lambda_ = lambda_
        
    def solve(self, h):
        x = self.body.currPos[self.id]
        n = x - self.rest_position
        d = np.linalg.norm(n)
        C = d
        if d < 1e-6:
            return
        alpha = self.compliance / h / h
        dlambda = - (C + alpha * self.lambda_) / (self.w + alpha)
        self.lambda_ += dlambda
        dx = dlambda * n / d
        self.body.currPos[self.id] += self.w * dx


class GroundCollisionConstraint(Constraint):
    def __init__(self, body, id, plane, compliance=0.00001, lambda_=0.0):
        self.body = body
        self.id = id # contact id
        self.w = body.inv_mass
        
        self.plane = plane
        self.compliance = compliance
        self.lambda_ = lambda_
        
    def solve(self, h):
        
        C = np.dot(self.body.currPos[self.id] - self.plane.origin, self.plane.normal)
        if C >= 0:
            return
        
        dC = self.plane.normal
            
        # Compliance
        alpha = self.compliance / h / h
        dlambda = - (C + alpha * self.lambda_) / (self.w + alpha)
        self.lambda_ += dlambda
        
        dx = dlambda * dC        
        
        
        
        self.body.currPos[self.id] += self.w * dx
        
            
    def solve_velocity(self, h):
        v = self.body.currVel[self.id]
        n = self.plane.normal
        
        v_n = np.dot(v, n) * n
        v_t = v - v_n
        
        self.body.currVel[self.id] = - v_n * self.body.restitution + v_t * self.body.friction



# Define PBD Simulation Class
Define the PBDSimulation class to handle the simulation steps, including integration, constraint solving, and collision handling.

In [5]:
class PBDSimulation:
    def __init__(self, 
                 time_step: float = 0.0333,  
                 substeps: int = 1,
                 gravity: list = None,
                 dynamic_collision: bool = False):
        """
        Initializes the PBD simulation environment.
        
        Args:
            time_step (float): The simulation time step.
            substeps (int): Number of substeps per time step for stability.
            gravity (list): Gravity vector (default: [0, -9.8, 0]).
        """
        self.dynamic_bodies: list[PhysicalObject] = []
        self.static_bodies: list[PhysicalObject] = []
        self.constraints: list[Constraint] = []

        self.dynamic_collisions = dynamic_collision
    
        self.gravity = np.array(gravity if gravity is not None else [0, -9.8, 0], dtype=np.float32)
        self.time_step = time_step
        self.substeps = max(1, substeps)
        self.h = self.time_step / self.substeps

    def add_constraint(self, constraint: Constraint):
        """Adds a constraint to the simulation."""
        self.constraints.append(constraint)

    def add_body(self, body):
        if isinstance(body, list):
            for b in body:
                self._add_single_body(b)
        else:
            self._add_single_body(body)

    def _add_single_body(self, body):
        if body.is_static:
            self.static_bodies.append(body)
        else:
            self.dynamic_bodies.append(body)        
    
    def step(self):
        collisions = self.check_collisions()
        
        for _ in range(self.substeps):
            contacts = self.check_contacts(collisions)
            self.integrate()
            self.solve_positions(contacts)
            self.update_velocities()
            self.solve_velocities(contacts)

    def check_collisions(self):
        if not self.dynamic_collisions:
            return []
        ## OBB - SAT
        potential_collisions = []
        
        # Get all cube vertices
        cube_positions = np.array([cube.currPos for cube in self.dynamic_bodies])  # Shape (N, 8, 3)

        # Compute face normals for each cube
        cube_faces = np.array([cube.faces for cube in self.dynamic_bodies])  # Shape (N, 12, 3)
        cube_faces = cube_faces[:, ::2] # Shape (N, 6, 3) : two faces have same normal vector
        
        q1, q2, q3 = cube_positions[:, cube_faces[:, :, 0]], cube_positions[:, cube_faces[:, :, 1]], cube_positions[:, cube_faces[:, :, 2]]
        face_normals = np.cross(q2 - q1, q3 - q1)  # Shape (N, 6, 3)
        face_normals /= np.linalg.norm(face_normals, axis=2, keepdims=True)

        # Compute edge directions for each cube
        cube_edges = np.array([cube.edges for cube in self.dynamic_bodies])  # Shape (N, 12, 2)
        edge_dirs = cube_positions[:, cube_edges[:, :, 1]] - cube_positions[:, cube_edges[:, :, 0]]
        edge_dirs /= np.linalg.norm(edge_dirs, axis=2, keepdims=True)

        # Compute cross products (Edge1 x Edge2) for all cube pairs
        cross_axes = np.cross(edge_dirs[:, :, None, :], edge_dirs[None, :, :, :])  # Shape (N, N, 12, 12, 3)
        valid_mask = np.linalg.norm(cross_axes, axis=4) > 1e-6  # Avoid zero vectors
        cross_axes[~valid_mask] = 0
        cross_axes /= np.linalg.norm(cross_axes, axis=4, keepdims=True, where=valid_mask)

        # Combine separating axes (face normals + cross products)
        separating_axes = np.concatenate((face_normals[:, None, :, :], face_normals[None, :, :, :], cross_axes), axis=2)  # Shape (N, N, 15, 3)

        # Project all cube vertices onto separating axes
        proj = np.einsum('nij, nmj->nim', cube_positions, separating_axes)  # Shape (N, N, 8, 15)

        # Get min and max projections
        min_proj, max_proj = np.min(proj, axis=2), np.max(proj, axis=2)  # Shape (N, N, 15)

        # Check for separating axis (vectorized SAT test)
        is_separated = (max_proj[:, :, None, :] < min_proj[None, :, :, :]) | (max_proj[None, :, :, :] < min_proj[:, :, None, :])
        collision_mask = ~np.any(is_separated, axis=-1)

        # Extract colliding pairs
        colliding_indices = np.column_stack(np.where(collision_mask))
        
        for i, j in colliding_indices:
            if i < j:
                potential_collisions.append((self.dynamic_bodies[i], self.dynamic_bodies[j]))

        return potential_collisions


    def check_contacts(self, collisions):
        contacts = []

        # Body-Body Collision
        if self.dynamic_collisions:
            for body1, body2 in collisions:
                contact_type, p1, p2, normal, penetration_depth = self.compute_dynamic_contacts(body1, body2)
                if penetration_depth <= 0:
                    continue  # No penetration → No contact constraint
                
                # **Dynamic-Dynamic Collision**
                if isinstance(body1, Cube) and isinstance(body2, Cube):
                    if contact_type == "vertex-face":
                        contacts.append(PointPlaneDistanceConstraint(body1, p1, body2, p2, normal, penetration_depth))
                    elif contact_type == "edge-edge":
                        contacts.append(EdgeEdgeDistanceConstraint(body1, p1, body2, p2, normal, penetration_depth))

        
        
        # Ground Collision
        plane = self.static_bodies[0].mesh
        for i, body in enumerate(self.dynamic_bodies):
            coll_idx = np.where(np.dot(body.currPos - plane.origin, plane.normal) < 0)[0]
            for i in coll_idx:
                contacts.append(GroundCollisionConstraint(body, i, plane,)) 
        return contacts
    
    
    def compute_dynamic_contacts(self, body1, body2):
        """Detects vertex-face and edge-edge contacts between two bodies."""
        
        ### **Step 1: Detect Vertex-Face Collisions**
        vertices = body1.currPos
        faces = body2.faces

        q1, q2, q3 = body2.currPos[faces[:, 0]], body2.currPos[faces[:, 1]], body2.currPos[faces[:, 2]]

        # Compute face normals
        e1, e2 = q2 - q1, q3 - q1
        face_normals = np.cross(e1, e2)
        face_normals /= np.linalg.norm(face_normals, axis=1, keepdims=True)

        # Compute signed distances (vertex penetration)
        v_minus_q1 = vertices[:, None, :] - q1[None, :, :]
        signed_distances = np.einsum('ijk,jk->ij', v_minus_q1, face_normals)

        # Find penetrating vertices
        penetration_mask = signed_distances < 0
        vertex_indices, face_indices = np.where(penetration_mask)

        if len(vertex_indices) > 0:
            # Select the deepest penetration
            min_penetration_idx = np.argmin(signed_distances[vertex_indices, face_indices])
            v_idx = vertex_indices[min_penetration_idx]
            f_idx = face_indices[min_penetration_idx]
            normal = face_normals[f_idx]
            penetration_depth = -signed_distances[v_idx, f_idx]

            return "vertex-face", v_idx, faces[f_idx], normal, penetration_depth

        ### **Step 2: Detect Edge-Edge Collisions**
        edges1, edges2 = body1.edges, body2.edges
        p1, p2 = body1.currPos[edges1[:, 0]], body1.currPos[edges1[:, 1]]
        q1, q2 = body2.currPos[edges2[:, 0]], body2.currPos[edges2[:, 1]]

        d1, d2 = p2 - p1, q2 - q1

        # Compute closest points between edges
        a = np.einsum('ij,ij->i', d1, d1)
        b = np.einsum('ij,ij->i', d1, d2)
        c = np.einsum('ij,ij->i', d2, d2)
        d = np.einsum('ij,ij->i', q1 - p1, d1)
        e = np.einsum('ij,ij->i', q1 - p1, d2)

        det = a * c - b * b
        s = np.clip((b * e - c * d) / det, 0, 1)
        t = np.clip((a * e - b * d) / det, 0, 1)

        closest_p1 = p1 + s[:, None] * d1
        closest_q1 = q1 + t[:, None] * d2
        distances = np.linalg.norm(closest_p1 - closest_q1, axis=1)

        # Find closest edge pair
        min_distance_idx = np.argmin(distances)
        penetration_depth = -distances[min_distance_idx]
        
        if penetration_depth < 0:
            return "edge-edge", edges1[min_distance_idx], edges2[min_distance_idx], (closest_p1[min_distance_idx] - closest_q1[min_distance_idx]) / np.linalg.norm(closest_p1[min_distance_idx] - closest_q1[min_distance_idx]), penetration_depth

        return None, None, None, None, None
    
    
    def integrate(self):
        for body in self.dynamic_bodies:
            body.prevPos = body.currPos.copy()
            body.currVel += self.gravity * self.h
            body.currPos += body.currVel * self.h


    def solve_positions(self, contacts):
        for constraint in self.constraints:
            constraint.solve(self.h)
        
        for contact in contacts:
            contact.solve(self.h)


    def update_velocities(self):
        for body in self.dynamic_bodies:
            body.currVel = (body.currPos - body.prevPos) / self.h


    def solve_velocities(self, contacts):
        for contact in contacts:
            contact.solve_velocity(self.h)
        
    
    def reset(self):
        for body in self.dynamic_bodies:
            body.reset()
        for constraint in self.constraints:
            constraint.lambda_ = 0.0
            
            
    def update(self):
        for body in self.dynamic_bodies:
            body.update_mesh()

# Initialize Simulation
Initialize the simulation by creating instances of Cube and Plane, and adding them to the PBDSimulation instance.

In [6]:
# Initialize Simulation
# Initialize the simulation by creating instances of Cube and Plane, and adding them to the PBDSimulation instance.

# Create instances of Cube and Plane
cube1 = PhysicalObject(Cube(width=1.0, height=2, depth=1, color='yellow', position=[0, 4.0, 0]), is_static=False)
cube2 = PhysicalObject(Cube(width=1, height=2, depth=1, color='blue', position=[0, 2, 0]), is_static=False)
cube3 = PhysicalObject(Cube(width=1, height=1, depth=1, color='red', position=[1.0, 0.5, 0], rotation=[0, 0, 90]), is_static=False)

plane = PhysicalObject(Plane(origin=[0, 0, 0], normal=[0, 1, 0], size=50, color='gray', opacity=1.0), is_static=True)

# Initialize PBDSimulation instance
pbd = PBDSimulation(
    time_step=0.0333,
    substeps=5,
    gravity=[0, -9.8, 0],
    dynamic_collision=False,
)

# Add bodies to the simulation
bodies = [cube1, cube2, cube3, plane]
pbd.add_body(bodies)

/home/matthew/anaconda3/lib/python3.12/site-packages/pythreejs/traits.py:257: UserWarning: 64-bit data types not supported for WebGL data, casting to 32-bit.
  warnings.warn('64-bit data types not supported for WebGL '


# Add Constraints
Add constraints to the simulation, such as distance constraints between cubes.

In [7]:
# Rigidbody Constraints
pbd.add_constraint(AttachmentConstraint(cube1, 2, [0.5, 5, 0.5]))

for cube in [cube1, cube2, cube3]:
    for (edge, rest_length) in zip(cube.edges, cube.rest_length):
        pbd.add_constraint(DistanceConstraint(cube, edge[0], cube, edge[1], rest_length))


# Run Simulation
Run the simulation by stepping through the simulation steps and updating the positions and velocities of the objects.

In [8]:
running = False


async def run_simulation():
    global running
    while running:
        pbd.step()
        for body in bodies:
            if not body.is_static:
                body.update_mesh()
        await asyncio.sleep(pbd.time_step)

# Function to start/stop the simulation
def toggle_simulation(_):
    global running
    running = not running
    if running:
        asyncio.create_task(run_simulation())

# Function to reset the simulation
def reset_simulation(_):
    global running
    running = False
    pbd.reset()

# Create buttons to control the simulation
start_stop_button = widgets.Button(description="START / STOP", button_style='success')
start_stop_button.on_click(toggle_simulation)

reset_button = widgets.Button(description="RESET", button_style='danger')
reset_button.on_click(reset_simulation)

buttons = widgets.HBox([reset_button, start_stop_button])

# Render Simulation
Render the simulation using pythreejs and ipywidgets to visualize the dynamic behavior of the objects.

In [9]:
# Rendering setup
width, height = 800, 600
aspect = width / height

scene = Scene(background='black')  # Background color of the scene
camera = PerspectiveCamera(
    position=[5, 5, 10], 
    lookAt=[0, 0, 0],
    aspect=aspect
)

ambient_light = AmbientLight(color='white', intensity=1.0)  # Soft overall light
directional_light = DirectionalLight(
    color='white',
    intensity=2.0,        # Brightness
    position=[0, 50, 100],   # Light direction
)
directional_light.castShadow = True  # Enable shadow casting

scene.add([ambient_light, directional_light, cube1.mesh, cube2.mesh, cube3.mesh, plane.mesh])

controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller], width=width, height=height)
renderer.shadowMap.enabled = True
renderer.shadowMap.type = 'PCFSoftShadowMap'

# Display the renderer
display(widgets.VBox([renderer, buttons]))

In [ ]:
while True:
    pbd.step()
    for body in bodies:
        if not body.is_static:
            body.update_mesh()
    time.sleep(pbd.time_step)